In [1]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [2]:
# preprocessing the json data
# tokenization
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# lemmatize, lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))


63 documents
17 classes ['Best Milege', 'Brands', 'Car specifications', 'EV manufacturers', 'EV model', 'EV model1', 'EV model2', 'Fuel tank capacity', 'Fuel_cng', 'Fuel_petrol', 'Model_price', 'SUV Cars', 'Safe cars', 'ev_model', 'ev_model2', 'ev_model3', 'model']
76 unique lemmatized words ['10', '5', 'audi', 'automotive', 'available', 'azure', 'bentley', 'best', 'between', 'brand', 'cadillac', 'capacity', 'car', 'chevrolet', 'chrysler', 'city', 'cng', 'common', 'company', 'consider', 'dodge', 'dynamic', 'electric', 'ev', 'fiat', 'fisker', 'ford', 'fuel', 'honda', 'hyundai', 'in', 'jaguar', 'jeep', 'kia', 'lakh', 'land', 'lincoln', 'lucid', 'manufacturer', 'max', 'maximum', 'mercedes-benz', 'mileage', 'min', 'mini', 'minimum', 'mitsubishi', 'model', 'most', 'motor', 'nissan', 'nk', 'of', 'petrol', 'polestar', 'porsche', 'preffered', 'rivian', 'rover', 'safe', 'safest', 'smart', 'specification', 'subaru', 'suv', 'tank', 'tata', 'tesla', 'th', 'to', 'toyota', 'vehicle', 'volkswagen', '

In [4]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [5]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
13/13 [==============================] - 1s 5ms/step - loss: 2.7833 - accuracy: 0.1429
Epoch 2/200
13/13 [==============================] - 0s 4ms/step - loss: 2.4386 - accuracy: 0.5238
Epoch 3/200
13/13 [==============================] - 0s 4ms/step - loss: 2.1270 - accuracy: 0.5079
Epoch 4/200
13/13 [==============================] - 0s 5ms/step - loss: 2.0072 - accuracy: 0.5079
Epoch 5/200
13/13 [==============================] - 0s 5ms/step - loss: 1.8705 - accuracy: 0.5556
Epoch 6/200
13/13 [==============================] - 0s 3ms/step - loss: 1.8055 - accuracy: 0.5397
Epoch 7/200
13/13 [==============================] - 0s 3ms/step - loss: 1.6720 - accuracy: 0.5397
Epoch 8/200
13/13 [==============================] - 0s 5ms/step - loss: 1.6344 - accuracy: 0.5556
Epoch 9/200
13/13 [==============================] - 0s 4ms/step - loss: 1.4901 - accuracy: 0.6032
Epoch 10/200
13/13 [==============================] - 0s 3ms/step - loss: 1.4153 - accuracy: 0.5873
Epoch 11/

13/13 [==============================] - 0s 2ms/step - loss: 0.1377 - accuracy: 0.9841
Epoch 84/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1704 - accuracy: 0.9683
Epoch 85/200
13/13 [==============================] - 0s 4ms/step - loss: 0.1718 - accuracy: 0.9365
Epoch 86/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1470 - accuracy: 0.9683
Epoch 87/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1941 - accuracy: 0.9841
Epoch 88/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1377 - accuracy: 0.9841
Epoch 89/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1865 - accuracy: 0.9365
Epoch 90/200
13/13 [==============================] - 0s 4ms/step - loss: 0.1158 - accuracy: 0.9683
Epoch 91/200
13/13 [==============================] - 0s 3ms/step - loss: 0.2252 - accuracy: 0.9524
Epoch 92/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1798 - accuracy: 0.9365
Epoch 93/200


13/13 [==============================] - 0s 4ms/step - loss: 0.0713 - accuracy: 0.9841
Epoch 165/200
13/13 [==============================] - 0s 3ms/step - loss: 0.0595 - accuracy: 0.9841
Epoch 166/200
13/13 [==============================] - 0s 3ms/step - loss: 0.0797 - accuracy: 0.9841
Epoch 167/200
13/13 [==============================] - 0s 3ms/step - loss: 0.0708 - accuracy: 0.9841
Epoch 168/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1046 - accuracy: 0.9683
Epoch 169/200
13/13 [==============================] - 0s 3ms/step - loss: 0.0800 - accuracy: 0.9683
Epoch 170/200
13/13 [==============================] - 0s 3ms/step - loss: 0.0644 - accuracy: 0.9841
Epoch 171/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1160 - accuracy: 0.9841
Epoch 172/200
13/13 [==============================] - 0s 3ms/step - loss: 0.1015 - accuracy: 0.9841
Epoch 173/200
13/13 [==============================] - 0s 3ms/step - loss: 0.0708 - accuracy: 0.9841
Epoc

In [6]:
# load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [8]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [9]:
# function to start the chat bot which will continue till the user type 'end'

def start_chat():
    print("please as your query regarding analysis.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [10]:
# start the chat bot
start_chat()

please as your query regarding analysis.


tesla 
1/1 [==============================] - 0s 202ms/step
Bot: MODEL 3,MODEL Y,MODEL X,MODEL S,ROADSTER

--------------------------------------------------
tesla cars
1/1 [==============================] - 0s 52ms/step
Bot: MODEL 3,MODEL Y,MODEL X,MODEL S,ROADSTER

--------------------------------------------------
hyundai cars
1/1 [==============================] - 0s 45ms/step
Bot: LEAF

--------------------------------------------------
hyundai cars
1/1 [==============================] - 0s 50ms/step
Bot: LEAF

--------------------------------------------------
tesla cars
1/1 [==============================] - 0s 50ms/step
Bot: MODEL 3,MODEL Y,MODEL X,MODEL S,ROADSTER

--------------------------------------------------
trending safest car
1/1 [==============================] - 0s 49ms/step
Bot:  Mercedes Benz, Nissan Rogue,Toyota Corolla, Acura TLX,BMW 

--------------------------------------------------
petrol cars available in market
1/